In [22]:
import warnings

warnings.filterwarnings('ignore')

In [20]:
%matplotlib inline

import pickle
import random as rnd
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import seaborn as sns
import sklearn
import statsmodels.api as sm
import xgboost as xgb
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from matplotlib import pyplot
from matplotlib.font_manager import FontProperties
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,
                                           QuadraticDiscriminantAnalysis)
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              ExtraTreesClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, VotingClassifier)
from sklearn.feature_selection import (RFE, SelectFromModel, SelectKBest,
                                       VarianceThreshold, chi2)
from sklearn.inspection import permutation_importance
from sklearn.linear_model import (LinearRegression, LogisticRegression, PassiveAggressiveClassifier,
                                  Perceptron, RidgeClassifier, SGDClassifier)
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, confusion_matrix, log_loss,
                             precision_score, recall_score, roc_auc_score)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     ShuffleSplit, StratifiedKFold,
                                     cross_val_score, train_test_split)
from sklearn.naive_bayes import (BernoulliNB, ComplementNB, GaussianNB,
                                 MultinomialNB)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (Binarizer, LabelEncoder, MinMaxScaler,
                                   Normalizer, OneHotEncoder,
                                   PolynomialFeatures, StandardScaler,
                                   normalize)
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from math import exp, log

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [11]:
df = pd.read_csv("../data/pretrain_label.csv")
df = df.drop(["arrival_date"], axis=1)

In [12]:
df["lnrevenue"] = df["revenue"].apply(lambda x: log(x))

In [13]:
df.head()

,label,revenue,lnrevenue
0,2.0,8133.6980,9.003771
1,1.0,3507.2961,8.162601
2,1.0,3643.1086,8.200593
3,1.0,3981.6194,8.289444
4,1.0,4046.5700,8.305625


In [14]:
new_train_x = df.drop(["label"], axis=1)
new_train_y = df["label"]

In [23]:
classifiers = [
               DecisionTreeClassifier(),
               XGBClassifier(),
               ExtraTreesClassifier(),
               RandomForestClassifier(),
               RidgeClassifier(),
               QuadraticDiscriminantAnalysis(),
               ComplementNB(),
               BernoulliNB(),
               GaussianNB(),
               MultinomialNB(),
               LinearDiscriminantAnalysis(),
               SGDClassifier(),
               AdaBoostClassifier(),
               BaggingClassifier(),
               LinearSVC(),
               KNeighborsClassifier(),
               XGBClassifier(),
               XGBRFClassifier(),
               LGBMClassifier()
               ]

from time import time

for clf in classifiers:
    
    before = time()
    print("="*30)
    print(clf.__class__.__name__)
    
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    crossValScore = cross_val_score(clf, new_train_x, new_train_y, scoring='neg_mean_absolute_error', cv=skf)  
    crossValScore_mean = abs(np.mean(crossValScore)) 
    print("mean_absolute_error: {:.4}".format(crossValScore_mean))
    after = time()
    print(f"花費{(after - before):.1f}s")
    
print("="*30)

DecisionTreeClassifier
mean_absolute_error: 0.6766
花費0.0s
XGBClassifier
mean_absolute_error: 0.5453
花費0.5s
ExtraTreesClassifier
mean_absolute_error: 0.6766
花費0.5s
RandomForestClassifier
mean_absolute_error: 0.6672
花費0.7s
RidgeClassifier
mean_absolute_error: 0.5875
花費0.0s
QuadraticDiscriminantAnalysis
mean_absolute_error: nan
花費0.0s
ComplementNB
mean_absolute_error: 0.8625
花費0.0s
BernoulliNB
mean_absolute_error: 1.086
花費0.0s
GaussianNB
mean_absolute_error: 0.4797
花費0.0s
MultinomialNB
mean_absolute_error: 0.5625
花費0.0s
LinearDiscriminantAnalysis
mean_absolute_error: 0.4953
花費0.0s
SGDClassifier
mean_absolute_error: 1.942
花費0.1s
AdaBoostClassifier
mean_absolute_error: 0.8078
花費0.4s
BaggingClassifier
mean_absolute_error: 0.6281
花費0.1s
LinearSVC
mean_absolute_error: 2.391
花費0.4s
KNeighborsClassifier
mean_absolute_error: 0.5219
花費0.0s
XGBClassifier
mean_absolute_error: 0.5469
花費0.5s
XGBRFClassifier
mean_absolute_error: 0.4984
花費0.4s
LGBMClassifier
mean_absolute_error: 0.5422
花費1.7s


In [26]:
X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.3)
model = XGBRFClassifier()
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Mean absolute error: {metrics.mean_absolute_error(y_test, y_pred):.2f}")

Mean absolute error: 0.53


In [56]:
df = pd.read_csv("../data/test.csv")
df["total_nights"] = df["stays_in_weekend_nights"] + df["stays_in_week_nights"]
df = df[df["total_nights"] > 0]
df["total_customers"] = df["adults"] + df["children"] + df["babies"]
df = df[df["total_customers"] <= 5]
df.shape

(27800, 31)

In [44]:
# testing_df = pd.read_csv("../data/test_data.csv")
testing_datedf = pd.read_csv("../data/test_nolabel.csv")
adr = pd.read_csv("../data/testing_adr_label.csv")

In [100]:
adr.describe()

,adr
count,27800.000000
mean,96.561608
std,35.505193
min,7.165013
25%,75.195244
50%,95.351570
75%,114.678452
max,287.254970


In [58]:
testingdf = df.copy()
testingdf["adr"] = adr.values
testingdf.head()

,ID,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,total_nights,total_customers,adr
0,91531,City Hotel,75,2017,April,13,1,2,5,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,0,1,7,2.0,94.976340
1,91532,City Hotel,208,2017,April,13,1,4,10,2,0.0,0,SC,BRA,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,0,1,14,2.0,60.524933
2,91533,Resort Hotel,12,2017,April,13,1,2,5,2,0.0,0,Undefined,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,40.0,NaN,0,Contract,0,1,7,2.0,75.688600
3,91534,City Hotel,76,2017,April,13,1,2,5,3,0.0,0,BB,BEL,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,0,0,7,3.0,113.164520
4,91535,City Hotel,9,2017,April,13,1,2,4,2,2.0,0,BB,FRA,Online TA,TA/TO,0,0,0,F,F,0,No Deposit,9.0,NaN,0,Transient,0,0,6,4.0,181.963560


In [60]:
def create_time_feature(df: pd.DataFrame, year: pd.Series, month: pd.Series, date: pd.Series) -> pd.Series:
    data = df.copy()
    data["date"] = pd.to_datetime(year.astype(str) + month + date.astype(str), format="%Y%B%d")
    data = pd.get_dummies(data, columns=["arrival_date_month"])
    data["weekday"] = data["date"].dt.weekday
    data["is_weekend"] = ((data["weekday"] == 5) | (data["weekday"] == 6)).astype("float")
    data = data.drop(["arrival_date_year", "arrival_date_day_of_month", "weekday"], axis=1)
    
    return data

testingdf = create_time_feature(testingdf, testingdf["arrival_date_year"], testingdf["arrival_date_month"], testingdf["arrival_date_day_of_month"])

In [92]:
newdf = testingdf[["date", "adr", "total_nights"]]
newdf["revenue"] = newdf["adr"] * newdf["total_nights"]

In [93]:
newdf = pd.DataFrame(newdf.groupby("date")["revenue"].agg("sum"))
newdf["lnrevenue"] = newdf["revenue"].apply(lambda x: log(x))
newdf

,revenue,lnrevenue
date,,
2017-04-01,44798.480753,10.709930
2017-04-02,33514.324712,10.419728
2017-04-03,61304.133196,11.023603
2017-04-04,31495.550305,10.357602
2017-04-05,51045.344677,10.840470
2017-04-06,56139.049901,10.935587
2017-04-07,36099.138932,10.494024
2017-04-08,69023.592968,11.142204
2017-04-09,93018.978642,11.440559


In [95]:
x_out = newdf
y_pred = model.predict(x_out)

In [97]:
y_pred

array([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])